# Regression model for Taxi fares using NimbusML

Regression is a ML task type of supervised machine learning algorithms. A regression ML model predicts continuous value outputs (such as numbers). For instance, predicting the fare of a Taxi trip or predicting the price of a car is a regression problem.

## Verify your NimbusML version

In [ ]:
import nimbusml
print( nimbusml.__version__)

## Load your data

In [ ]:
from nimbusml import FileDataStream

# these are actually faster than Pandas Dataframes
# Do you want to know more? https://arxiv.org/pdf/1905.05715.pdf
ds_train = FileDataStream.read_csv("./datasets/taxi/taxi-fare-train.csv")
ds_test = FileDataStream.read_csv("./datasets/taxi/taxi-fare-test.csv")
ds_train.head(5)

## Verify the schema of your data 

In [ ]:
# Want to know more about the Schema?
# https://docs.microsoft.com/en-us/nimbusml/concepts/schema

# Want to know more about data types? in Nimbus?
# https://docs.microsoft.com/en-us/nimbusml/concepts/types
ds_train.schema

## Data transformations pipeline for NimbusML model

In [ ]:
from nimbusml import Pipeline
from nimbusml.feature_extraction.categorical import OneHotVectorizer
from nimbusml.preprocessing.normalization import MeanVarianceScaler

# https://docs.microsoft.com/en-us/python/api/nimbusml/nimbusml.feature_extraction.categorical.onehotvectorizer?view=nimbusml-py-latest
onv = OneHotVectorizer()        <<['vendor_id','rate_code','payment_type']
mvs = MeanVarianceScaler()      <<['passenger_count','trip_time_in_secs','trip_distance']

preprocess_pipeline = Pipeline([onv,mvs])

## Make it a training pipeline

In [ ]:
from nimbusml.linear_model import OrdinaryLeastSquaresRegressor

olsr = OrdinaryLeastSquaresRegressor(feature= ['vendor_id', 'rate_code', 'passenger_count', 'trip_time_in_secs','trip_distance', 'payment_type'] , label='fare_amount')

training_pipeline = preprocess_pipeline.clone()

In [ ]:
training_pipeline.append(olsr)

## Fit both pipelines

In [ ]:
# fit the pipelines
preprocess_pipeline.fit(ds_train, 'y')
training_pipeline.fit(ds_train, 'y')

print(preprocess_pipeline)
print(training_pipeline)

## Observe the transformed data

In [ ]:
preprocess_pipeline.transform(ds_train)

## Measure the performance on the training and test set

In [ ]:
metrics, scores = training_pipeline.test(ds_train, output_scores=True)
#print(scores) # uncomment this if you want to look at the scores
metrics

In [ ]:
metrics, scores = training_pipeline.test(ds_test, output_scores=True)
#print(scores) # uncomment this if you want to look at the scores
metrics

## Visualize the pipeline

In [ ]:
###Does this not work?
### make sure you've installed graphviz https://graphviz.gitlab.io/download/
### make sure you've run set_path_graphviz.bat 
from nimbusml.utils.exports import img_export_pipeline
figure = img_export_pipeline(training_pipeline,ds_train)
figure